In [2]:
import json
from datetime import datetime, timedelta
from openai import OpenAI
import time
import sys

class AIWeatherPredictor:
    def __init__(self, openrouter_api_key):
        """
        Initialize the AI Weather Prediction System

        Args:
            openrouter_api_key (str): Your OpenRouter API key
        """
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=openrouter_api_key,
        )

    def get_location_info(self, location):
        """Get basic location information for better weather analysis"""
        location_prompt = f"""
Provide geographic and climate information for {location} including:
- Country and region
- Latitude/longitude (approximate)
- Elevation
- Climate zone
- Seasonal weather patterns
- Current month considerations
- Local geography that affects weather

Format as JSON with keys: country, region, lat, lon, elevation, climate_zone, seasonal_info, current_month_typical
"""

        try:
            completion = self.client.chat.completions.create(
                model="deepseek/deepseek-r1:free",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a geographic and climate expert. Provide accurate location and climate data in JSON format."
                    },
                    {
                        "role": "user",
                        "content": location_prompt
                    }
                ],
                temperature=0.1,
                max_tokens=500
            )

            # Try to parse JSON response
            response = completion.choices[0].message.content
            if '```json' in response:
                json_part = response.split('```json')[1].split('```')[0]
            elif '{' in response:
                json_part = response[response.find('{'):response.rfind('}')+1]
            else:
                json_part = response

            return json.loads(json_part)

        except Exception as e:
            print(f"⚠️ Could not get location info: {e}")
            return {"location": location, "info": "Limited location data available"}

    def create_weather_prediction_prompt(self, location, hours_ahead=24, location_info=None):
        """Create a comprehensive weather prediction prompt"""
        current_time = datetime.now()
        target_time = current_time + timedelta(hours=hours_ahead)

        season = self.get_current_season()

        prompt = f"""
You are an expert meteorologist with access to advanced atmospheric modeling and satellite data.
Provide a detailed weather prediction for {location} for the next {hours_ahead} hours.

CURRENT CONDITIONS:
- Current Date/Time: {current_time.strftime('%Y-%m-%d %H:%M:%S')}
- Target Prediction Time: {target_time.strftime('%Y-%m-%d %H:%M:%S')}
- Current Season: {season}

LOCATION INFORMATION:
{json.dumps(location_info, indent=2) if location_info else f"Location: {location}"}

Please provide a comprehensive weather forecast including:

1. **CURRENT WEATHER ANALYSIS**:
   - Temperature (°C/°F)
   - Weather conditions (sunny, cloudy, rainy, etc.)
   - Wind speed and direction
   - Humidity levels
   - Atmospheric pressure
   - Visibility conditions

2. **{hours_ahead}-HOUR DETAILED FORECAST**:
   - Hour-by-hour temperature trends
   - Precipitation probability and type
   - Wind pattern changes
   - Cloud cover evolution
   - Humidity and pressure trends
   - Any weather system movements

3. **WEATHER ALERTS & WARNINGS**:
   - Severe weather potential
   - Temperature extremes
   - Strong wind advisories
   - Precipitation warnings
   - Air quality considerations

4. **CONFIDENCE ASSESSMENT**:
   - Prediction confidence (1-10 scale)
   - Factors affecting accuracy
   - Most/least certain aspects

5. **PRACTICAL RECOMMENDATIONS**:
   - Clothing suggestions
   - Outdoor activity advice
   - Travel conditions
   - Agricultural implications
   - Health considerations

6. **EXTENDED OUTLOOK** (next 3-5 days):
   - General weather pattern
   - Temperature trends
   - Significant weather changes

Use current meteorological knowledge, seasonal patterns, geographic factors, and atmospheric science principles to make accurate predictions. Consider local topography, proximity to water bodies, urban heat effects, and seasonal weather patterns typical for this location and time of year.

Format your response clearly with sections and bullet points where appropriate.
"""
        return prompt

    def get_current_season(self):
        """Determine current season based on date"""
        month = datetime.now().month
        if month in [12, 1, 2]:
            return "Winter"
        elif month in [3, 4, 5]:
            return "Spring"
        elif month in [6, 7, 8]:
            return "Summer"
        else:
            return "Autumn"

    def get_ai_weather_prediction(self, prompt):
        """Get weather prediction from AI model"""
        try:
            print("🤖 Analyzing atmospheric conditions with AI...")
            completion = self.client.chat.completions.create(
                extra_headers={
                    "HTTP-Referer": "https://ai-weather-predictor.com",
                    "X-Title": "AI Weather Prediction System",
                },
                model="deepseek/deepseek-r1:free",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a world-class meteorologist with decades of experience in weather prediction, atmospheric science, and climate analysis. You have access to the latest meteorological models and satellite data. Provide detailed, accurate weather forecasts based on scientific principles, seasonal patterns, and geographic factors."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=0.2,  # Lower temperature for more consistent predictions
                max_tokens=3000
            )
            print("✅ AI weather analysis completed")
            return completion.choices[0].message.content
        except Exception as e:
            print(f"❌ Error getting AI prediction: {e}")
            print(f"🔍 Error type: {type(e).__name__}")

            # Check for common OpenRouter errors
            if "401" in str(e):
                print("💡 Check your OpenRouter API key - it may be invalid")
            elif "429" in str(e):
                print("💡 Rate limit exceeded - please wait a few minutes")
            elif "500" in str(e):
                print("💡 OpenRouter server error - try again in a moment")
            elif "insufficient" in str(e).lower():
                print("💡 Insufficient credits - check your OpenRouter account balance")

            return None

    def predict_weather(self, location, hours_ahead=24):
        """Main method to generate weather predictions"""
        print(f"🌤️ Generating AI weather prediction for {location}...")
        print(f"⏰ Forecast period: Next {hours_ahead} hours")
        print(f"📡 Getting location analysis...")

        # Get location information for better predictions
        location_info = self.get_location_info(location)

        print(f"🧠 Creating detailed meteorological analysis...")

        # Create prediction prompt
        prompt = self.create_weather_prediction_prompt(location, hours_ahead, location_info)

        # Get AI prediction
        prediction = self.get_ai_weather_prediction(prompt)

        if prediction:
            print(f"✅ Weather prediction generated successfully!")
            return prediction
        else:
            return "❌ Failed to generate AI prediction"

    def get_weather_comparison(self, location1, location2, hours_ahead=24):
        """Compare weather between two locations"""
        comparison_prompt = f"""
As a meteorologist, compare the weather conditions between {location1} and {location2} for the next {hours_ahead} hours.

Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Provide:
1. **Current conditions comparison**
2. **{hours_ahead}-hour forecast for each location**
3. **Key differences and similarities**
4. **Which location has better conditions for**:
   - Outdoor activities
   - Travel
   - Photography
   - Sports events
5. **Weather risks for each location**

Format clearly with sections for each location and comparison points.
"""

        try:
            completion = self.client.chat.completions.create(
                model="deepseek/deepseek-r1:free",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a comparative meteorology expert specializing in multi-location weather analysis."
                    },
                    {
                        "role": "user",
                        "content": comparison_prompt
                    }
                ],
                temperature=0.2,
                max_tokens=2500
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(f"❌ Error getting weather comparison: {e}")
            return None

    def continuous_monitoring(self, location, update_interval_minutes=60, duration_hours=24):
        """Continuously monitor and update weather predictions"""
        print(f"🔄 Starting continuous AI weather monitoring for {location}")
        print(f"📅 Update interval: {update_interval_minutes} minutes")
        print(f"⏰ Duration: {duration_hours} hours")
        print("-" * 60)

        end_time = datetime.now() + timedelta(hours=duration_hours)
        update_count = 0

        while datetime.now() < end_time:
            update_count += 1
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n🕒 Update #{update_count} at {current_time}")

            prediction = self.predict_weather(location, hours_ahead=24)
            print(f"\n📋 WEATHER PREDICTION UPDATE:")
            print("=" * 70)
            print(prediction)
            print("\n" + "="*70 + "\n")

            if datetime.now() < end_time:
                print(f"⏳ Next update in {update_interval_minutes} minutes...")
                time.sleep(update_interval_minutes * 60)

        print("✅ Continuous monitoring completed!")

def main():
    print("🌟 AI WEATHER PREDICTION SYSTEM 🌟")
    print("=" * 50)
    print("🚀 Powered by DeepSeek AI - No External APIs Required!")
    print()

    # Configuration
    OPENROUTER_API_KEY = "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"

    # Validate API key
    if not OPENROUTER_API_KEY or OPENROUTER_API_KEY.startswith("<YOUR"):
        print("❌ Please set your OpenRouter API key in the code")
        print("💡 Get your key at: https://openrouter.ai")
        sys.exit(1)

    # Initialize the AI weather predictor
    try:
        weather_predictor = AIWeatherPredictor(OPENROUTER_API_KEY)
        print("✅ AI Weather Predictor initialized successfully")
    except Exception as e:
        print(f"❌ Failed to initialize predictor: {e}")
        sys.exit(1)

    print("\n🎯 Choose your weather prediction option:")
    print("1. Single location forecast")
    print("2. Compare two locations")
    print("3. Continuous monitoring")

    try:
        choice = input("\nEnter your choice (1-3): ").strip()

        if choice == "1":
            # Single location prediction
            location = input("🌍 Enter location (city, country): ").strip()
            if not location:
                location = "New York, USA"
                print(f"Using default: {location}")

            hours = input("⏰ Forecast hours (default 24): ").strip()
            hours_ahead = int(hours) if hours.isdigit() else 24

            print(f"\n🎯 Generating prediction for: {location}")
            print("-" * 50)

            prediction = weather_predictor.predict_weather(location, hours_ahead)

            if prediction and "❌" not in prediction:
                print(f"\n📋 AI WEATHER PREDICTION FOR {location.upper()}:")
                print("=" * 60)
                print(prediction)
            else:
                print("❌ Failed to generate weather prediction")

        elif choice == "2":
            # Compare two locations
            location1 = input("🌍 Enter first location: ").strip()
            location2 = input("🌍 Enter second location: ").strip()

            if not location1 or not location2:
                print("❌ Both locations are required for comparison")
                sys.exit(1)

            hours = input("⏰ Forecast hours (default 24): ").strip()
            hours_ahead = int(hours) if hours.isdigit() else 24

            print(f"\n🆚 Comparing weather: {location1} vs {location2}")
            print("-" * 50)

            comparison = weather_predictor.get_weather_comparison(location1, location2, hours_ahead)

            if comparison:
                print(f"\n📋 WEATHER COMPARISON:")
                print("=" * 60)
                print(comparison)
            else:
                print("❌ Failed to generate weather comparison")

        elif choice == "3":
            # Continuous monitoring
            location = input("🌍 Enter location for monitoring: ").strip()
            if not location:
                location = "New York, USA"
                print(f"Using default: {location}")

            try:
                interval = int(input("⏰ Update interval in minutes (default 60): ") or "60")
                duration = int(input("📅 Monitoring duration in hours (default 12): ") or "12")

                weather_predictor.continuous_monitoring(location, update_interval_minutes=interval, duration_hours=duration)

            except KeyboardInterrupt:
                print("\n⏹️ Monitoring stopped by user")
            except ValueError:
                print("❌ Invalid input - using defaults")
                weather_predictor.continuous_monitoring(location, update_interval_minutes=60, duration_hours=12)

        else:
            print("❌ Invalid choice. Please run the program again.")

    except KeyboardInterrupt:
        print("\n👋 Program interrupted by user. Goodbye!")
    except Exception as e:
        print(f"❌ An error occurred: {e}")
        print("🔧 Please check your API key and internet connection")

if __name__ == "__main__":
    main()


🌟 AI WEATHER PREDICTION SYSTEM 🌟
🚀 Powered by DeepSeek AI - No External APIs Required!

✅ AI Weather Predictor initialized successfully

🎯 Choose your weather prediction option:
1. Single location forecast
2. Compare two locations
3. Continuous monitoring

Enter your choice (1-3): 1
🌍 Enter location (city, country): nashik india
⏰ Forecast hours (default 24): 24

🎯 Generating prediction for: nashik india
--------------------------------------------------
🌤️ Generating AI weather prediction for nashik india...
⏰ Forecast period: Next 24 hours
📡 Getting location analysis...
⚠️ Could not get location info: Unterminated string starting at: line 18 column 26 (char 482)
🧠 Creating detailed meteorological analysis...
🤖 Analyzing atmospheric conditions with AI...
✅ AI weather analysis completed
✅ Weather prediction generated successfully!

📋 AI WEATHER PREDICTION FOR NASHIK INDIA:
### **Weather Forecast for Nashik, India (2025-08-24 15:11 to 2025-08-25 15:11)**  
**Location**: Nashik, Maharash